# Introduction to R Programming: Loops, Conditionals, & Functions

Matthew D. Turner, PhD  
Georgia State University

Some rights reserved: [cc by-nc-sa](https://creativecommons.org/licenses/by-nc-sa/4.0/) See bottom of document for details.
***
## Example: Comparing Floating Point Numbers
This example uses conditionals, comparisons, and functions to solve a general problem in working with approximations.

All scientific measurements are **approximate** to some degree. They are approximate due to measurement error of various forms. 

For instance, there is the problem of _resolution_, which refers to the limits of an instrument to measure something. A tape measure used to measure a room will usually be accurate to about a tenth of an inch, while a precise laser measurement may be accurate to a hundredth of an inch. This is an example of two measurement tools that differ in resolution. However, measuring the size of a room rarely needs to be more accurate than, maybe, half and inch, so the laser might be more than in needed. When making scientific measurements, _the level of accuracy required is a decision for us to make_. When measuring the distance to the sun, do we need to know it down to the millimeter, or the meter, or the kilometer?

Perhaps the measurement suffers from an _accuracy_ problem? Maybe the accuracy problem is _definitional_? What is the distance to the sun from the earth? Does this vary as the planet changes position in its orbit? [Can we define the distance from earth to sun in a meaningful way?](https://en.wikipedia.org/wiki/Astronomical_unit) Clearly there is no single number that can capture this concept completely. There are also accuracy problems related to repeated measurements: what if we measure a distance precisely with a laser many times, do we expect to always get identical answers? Probably not. The same idea holds for our scientific instrumnents.

Finally there is the computer representation of numbers. This is a concept a little too advanced for this class, but computers can only represent numbers up to some limits of accuracy. For most problems and most situations, we do not need to worry about this too much, but we should always be aware that **any** (decimal) number in a computer is limited in how exact it is, no matter what.

### The Problem of Equality
Because of the considerations above, the direct comparison of two numbers, even numbers that are supposed to be identical, may fail. We had an example from elsewhere in this course that raised this issue, let's return to that now.

In an earlier notebook, we computed the standard deviation (`sd`) and variance (`var`) of the weight variable from our height and weight data set (`height_weight_200.csv`). We got the `TRUE` answer we were looking for, but I told you it might not work. Why?

In [ ]:
hw <- read.csv("height_weight_200.csv")
sd(hw$weight) == sqrt(var(hw$weight))    # Get the square root of the variance

From your statistics class you should have learned that: $\mathrm{s.d.} = \sqrt{\mathrm{variance}}$, or that the standard deviation is the square root of variance. So it should be the case that: $(\mathrm{s.d.})^2 = \mathrm{variance}$. Mathematically this **is** true.

Above, we compared the square root of the variance with the standard deviation. It turns out that R, _internally_, does exactly that calculation to get the standard deviation. But what if we went the other way and squared the standard deviation?

In [ ]:
sd(hw$weight)^2 == var(hw$weight)       # Here we **square** the sd instead

Why? Let's look at the numbers:

In [ ]:
sqrt(var(hw$weight))
sd(hw$weight)

In [ ]:
sd(hw$weight)^2
var(hw$weight)

Wait, what? That looks ok to me! What is going on here?

### Another Way to Check Equality
If two numbers are the same, then what happens if I subtract one from the other? Seriously, think for a moment. If I have 3.145 and 3.145, then 3.145 - 3.145 = 0. But what if I have 3.141592 and 3.141593? Both of these are very good approximations of $\pi$, but even though they are nearly the same, what is their difference?

Let's see if we can use this to discover what is different in the examples above. For both pairs of numbers above, we get the difference.

In [ ]:
sqrt(var(hw$weight)) - sd(hw$weight)

So far, so good...

In [ ]:
sd(hw$weight)^2 - var(hw$weight)

What is going on? Well, the two numbers are very close together, so close that in the first 12 decimal places they look **exactly the same** (see above). But somewhere further on they differ by a little bit. At least they do in the case where we **square the `sd` result**. Specifically, by subtracting we can see that they differ by an amount on the order of $10^{-14}$ or $0.00000000000001$. (Remember that in R's scientific notation, the e-14 part represents the $10^{-14}$ part from your math and science classes. You may have seen the number R gives you above, `-2.8421709430404e-14` written as $-2.8421709430404 \times 10^{-14}$ in your math, physics, or science classes.)

> **Aside:** We will not continue with this topic in this class, but this is an example from the field of **numerical mathematics** or **numerical analysis** the branch of applied mathematics that studies the propagation of small errors in mathematical calculations (among many other things!). It is also the study of computer representations of numbers. 

What you need to know for now is this: never directly check to see if two _floating point_ numbers are equal with `==` in R. (Or any other computational environment.)

The general solution for this sort of problem is to define an **error tolerance**, usually called $\tau$ (or sometimes $\epsilon$) and agree that when two numbers are closer together than $\tau$ they are the same. Then we just pick a $\tau$ small enough for our application and we are fine.

Specifically, for numbers $x$ and $y$, we will check to see if $\left|\,x - y\,\right| < \tau$, and when this is `TRUE` we will say that $x = y$. The only additional thing here is the absolute value, which is just to make sure the order in which we do the subtraction does not matter. (We need the error to be positive so that when we compare we have  $0 < \left|\,x - y\,\right| < \tau$. If we allowed the $x - y$ part to be negative then any sized difference would be less than $\tau$. If that is hard for you, just accept the definition for now, it is what professionals tell us to use!)

So what we need is a new function, that we will call `fp.equals` for "floating point equals" that does the same job as `==` but with a tolerance:

In [ ]:
fp.equals <- function(x, y, tau){
    d <- abs(x - y)
    if (d < tau){
        return(TRUE)
    } else {
        return(FALSE)
    } 
}

### Tests
The following lines are examples of **tests**. In some computer languages there are built-in _assertions_ which are commands that let you write simple test examples to check your code. R allows these by using libraries and packages, but does not have them built-in.  The following examples use your `fp.equals` function to test the cases we considered above and see that it works.

In [ ]:
fp.equals(sd(hw$weight)^2, var(hw$weight), tau = 0.000001) # Should be TRUE

In [ ]:
fp.equals(sd(hw$weight)^2, var(hw$weight), tau = 0.00000000000001) # From above; FALSE

### What's the Point?
The point of this example is to show how we can use two of the constructs from todays class to solve a problem computationally. Specifically,

+ We used _built-in_ functions (`abs`) and comparisons (`<`) to create, then compare the numbers.
+ We used the if-else construct to carry out the comparison between the difference of the two numbers and $\tau$ and to select the correct response.
+ We created our own function to wrap up this computation in a way that allows it to be used (re-used) in other contexts. Whenever we need to compare floating point numbers this is available to us to use, just by loading the function definition.

Additionally, it is important for all computer work in science to know that we are **always** dealing with approximations in our measurements and floating-point numbers, and therefore need to be careful that we use some basic concepts like **equality**  in an appropriately nuanced way.

***
Version 1.0  
2018.07.11

To contact the author, email [mturner46@gsu.edu](mailto:mturner46@gsu.edu). Please contact me with recommendations for improvement or if you find any errors. This work may be adapted for any non-commercial purpose within the bounds of the license.

<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.